In [507]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import time
import os
import gc
import nltk

np.random.seed(42)

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [508]:
df = pd.read_csv('products_sentiment_train.tsv', delimiter='\t', header=None, names=['Reviews', 'Target'])
df.head(5)

,Reviews,Target
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [509]:
df.isnull().values.any()

False

In [510]:
print ('Number of docs in session: {}'.format(len(df)))
np.bincount(review_labels)

Number of docs in session: 2000


array([ 726, 1274])

In [511]:
# оп, у нас несбалансированная по таргету выборка. Сделаем stratify по таргету в train_test_split как минимум.
# Будем пробовать SVM
# Еще добавим наивный байесовский классификатор ну и конечно RF :)
# Где сможем, попробуем class_weight='balanced', штрафующий меньше за ошибки в минорном классе.

In [512]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [513]:
X_train = df['Reviews']
y = df['Target']

In [514]:
X_test = pd.read_csv('products_sentiment_test.tsv', sep='\t', usecols=[1])
X_test.head(5)

,text
0,"so , why the small digital elph , rather than ..."
1,3/4 way through the first disk we played on it...
2,better for the zen micro is outlook compatibil...
3,6 . play gameboy color games on it with goboy .
4,"likewise , i 've heard norton 2004 professiona..."


In [515]:
X_test = X_test['text']

In [460]:
cv = CountVectorizer()
X = cv.fit_transform(reviews)

In [203]:
LR = LogisticRegression(random_state=42, n_jobs=-1)

In [214]:
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

In [461]:
np.mean(cross_val_score(LR, X, y, scoring = 'accuracy', cv=rskf))

0.7668011681323008

In [462]:
svm = SVC(kernel='linear', random_state=42)
np.mean(cross_val_score(svm, X, y, scoring = 'accuracy', cv=rskf))

0.7491495034343965

In [224]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB

In [463]:
MNB = MultinomialNB()
np.mean(cross_val_score(MNB, X, y, scoring = 'accuracy', cv=rskf))

0.7794979449871562

In [229]:
CNB = ComplementNB()
np.mean(cross_val_score(CNB, X, y, scoring = 'accuracy', cv=rskf))

0.7775475652972831

In [233]:
knc = KNeighborsClassifier(n_neighbors=4, n_jobs=-1)
np.mean(cross_val_score(knc, X, y, scoring = 'accuracy', cv=5))

0.6624853905336908

In [378]:
sgd = SGDClassifier(random_state=42, n_jobs=-1, loss='hinge', penalty='l2')
np.mean(cross_val_score(sgd, X, y, scoring = 'accuracy', cv=rskf))

0.7421381074256714

In [287]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(reviews)

In [235]:
MNB = MultinomialNB()
np.mean(cross_val_score(MNB, X, y, scoring = 'accuracy', cv=rskf))

0.7041493865586658

In [ ]:
#наивный байесовский ожидаемо хуже с TFIDF

In [236]:
svm = SVC(kernel='linear', random_state=42)
np.mean(cross_val_score(svm, X, y, scoring = 'accuracy', cv=rskf))

0.7688974299839373

In [288]:
np.std(cross_val_score(svm, X, y, scoring = 'accuracy', cv=rskf))

0.020652658703663684

In [237]:
np.mean(cross_val_score(LR, X, y, scoring = 'accuracy', cv=rskf))

0.7666507946924668

In [243]:
RF = RandomForestClassifier()
np.mean(cross_val_score(RF, X, y, scoring = 'accuracy', cv=rskf))

0.7101540281501759

In [ ]:
#переберем гиперпараметры для svm и посмотрим, что сможем выжать из NB с countvectorizer-ом

In [194]:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.8, stratify=y)

In [459]:
PipeNB = Pipeline([
        ('vectorizer', CountVectorizer()),
        ('mnb', MultinomialNB())
    ])

In [282]:
from nltk.corpus import stopwords

In [284]:
param_grid = [{'vectorizer__min_df':[1, 2, 3, 4, 5],
               'vectorizer__ngram_range':[(1, 2),(1, 3)]
              }]
GS = GridSearchCV(PipeNB, param_grid = param_grid, cv=5)
GS.fit(reviews, y)
GS.best_params_, GS.best_score_

({'vectorizer__min_df': 2, 'vectorizer__ngram_range': (1, 2)}, 0.7865)

In [291]:
PipeSVM = Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('svm', SVC(random_state=42))
    ])

In [292]:
param_grid = [{'vectorizer__min_df':[1, 2, 3, 4, 5],
               'vectorizer__ngram_range':[(1, 2),(1, 3)],
               'svm__C':[0.1, 0.5, 1],
               'svm__kernel':['linear', 'poly', 'rbf']
              }]
GS = GridSearchCV(PipeSVM, param_grid = param_grid, cv=5)
GS.fit(reviews, y)
GS.best_params_, GS.best_score_

({'svm__C': 1,
  'svm__kernel': 'linear',
  'vectorizer__min_df': 1,
  'vectorizer__ngram_range': (1, 3)},
 0.7835)

In [458]:
PipeSVM = Pipeline([
        ('vectorizer', TfidfVectorizer(analyzer='char_wb', min_df=2, stop_words='english')),
        ('svm', SVC(kernel='linear', random_state=42, C=0.5))
    ])
param_grid = {'vectorizer__ngram_range':[(2,5), (3,5)]}
GS = GridSearchCV(PipeSVM, param_grid = param_grid, cv=5)
GS.fit(reviews, y)
GS.best_params_, GS.best_score_

({'vectorizer__ngram_range': (2, 5)}, 0.7915)

In [328]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [338]:
En_stemmer = SnowballStemmer('english')

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([En_stemmer.stem(w) for w in analyzer(doc)])

In [339]:
PipeNB = Pipeline([
        ('vectorizer', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words='english')),
        ('mnb', MultinomialNB())
    ])
PipeSVM = Pipeline([
        ('vectorizer', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words='english')),
        ('svm', SVC(kernel='linear', random_state=42, C=0.5))
    ])

In [334]:
param_grid = {'vectorizer__ngram_range':[(1, 2),(1, 3)]}
GS = GridSearchCV(PipeSVM, param_grid = param_grid, cv=5)
GS.fit(reviews, y)
GS.best_params_, GS.best_score_

({'vectorizer__ngram_range': (1, 3)}, 0.7295)

In [340]:
param_grid = {'vectorizer__ngram_range':[(1, 2),(1, 3)]}
GS = GridSearchCV(PipeNB, param_grid = param_grid, cv=5)
GS.fit(reviews, y)
GS.best_params_, GS.best_score_

({'vectorizer__ngram_range': (1, 3)}, 0.7585)

In [ ]:
PipeSVM = Pipeline([
        ('vectorizer', TfidfVectorizer(analyzer='char_wb', min_df=2, stop_words='english')),
        ('svm', SVC(kernel='linear', random_state=42, C=0.5))
    ])
param_grid = {'vectorizer__ngram_range':[(2,5), (3,5)]}
GS = GridSearchCV(PipeSVM, param_grid = param_grid, cv=5)
GS.fit(reviews, y)
GS.best_params_, GS.best_score_

In [410]:
PipeSVM = Pipeline([
        ('vectorizer', TfidfVectorizer(analyzer='char_wb', min_df=2, ngram_range=(2,5), stop_words='english')),
        ('svm', SVC(kernel='linear', random_state=42, C=0.5))
    ])

In [412]:
PipeSVM.fit(reviews, y)
prediction_svm_charwb = PipeSVM.predict(X_test)

In [415]:
submission = pd.read_csv('products_sentiment_sample_submission.csv')
submission.head(5)

,Id,y
0,0,0
1,1,1
2,2,0
3,3,1
4,4,0


In [417]:
submission = pd.read_csv('products_sentiment_sample_submission.csv')
submission['y'] = prediction_svm_charwb
submission.to_csv('svm_charwb.csv', index = None)

submission.head()

,Id,y
0,0,1
1,1,0
2,2,1
3,3,1
4,4,0


In [ ]:
# годится, в след серии LTSM на keras

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 

Image(url= "https://i110.fastpic.ru/big/2019/0530/a6/_55bc96440f6ec48341b35d8de79da2a6.png?noht=1")
